# Create and Run a Discovery Agent

A Discovery Agent ingests, analyzes, and stores metadata about the data related assets in a data connection.  Agents are especially useful when metadata is not ingested in some other way, through events or via the catalog API.  The catalog service enables constructing, running, and monitoring Agents as independent, event driven processes. Agents store the metadata they collect into the catalog.  

## Imports and Global Variables

Run this cell before any of the others as it imports packages and sets variables that will be used throughout the notebook

In [ ]:
import requests
import json
sasserver = "https://your_server"
print (sasserver)

## Get Access token

In [ ]:
url = sasserver + "/SASLogon/oauth/token#password"

payload={
    'grant_type': 'password',
    'username': 'your_username',
    'password': 'your_password'
}
files=[

]
headers = {
  'Authorization': 'Basic c2FzLmVjOg==',
}

response = requests.request("POST", url, headers=headers, data=payload, files=files, verify=False)
print(response)
response = response.json()
print(response)
access_token = response["access_token"]

## Create Discovery Agent

In [ ]:
url = sasserver + "/catalog/bots"

payload = json.dumps({
  "provider": "TABLE-BOT",
  "name": "Samples Discovery Agent",
    "description": "Ingest, analyze, and store metadata from Samples caslib",
  "parameters": {
    "datasourceURI": "/dataSources/providers/cas/sources/cas-shared-default~fs~Samples",
    "region": "US NC"
  }
})
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)
print(response)
response = response.json()
print(response)
bot_id = response["id"]




## Run Discovery Agent

In [ ]:
url = sasserver + "/catalog/bots/" + bot_id + "/state?value=running"

payload = {}
headers = {
  'Accept': 'text/plain',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("PUT", url, headers=headers, data=payload, verify=False)
print(response)
print(response.text)


## Get Discovery Agent state

Repeat this code snippet until the response is idle.

In [ ]:
url = sasserver + "/catalog/bots/" + bot_id + "/state"

payload = {}
headers = {
  'Accept': 'text/plain',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)
print(response)
print(response.text)


## Post /instances#archive

This request will return instances with metadata from the Samples Library that was analyzed by the Discovery Agent.

In [ ]:
url = sasserver + "/catalog/instances"

payload = json.dumps({
  "query": "match (l:casLibrary {name:\"Samples\"})-[r:dataStoreDataSets]->(t:casTable)-[rc:dataSetDataFields]->(c:casColumn) return t,rc,c"
})
headers = {
  'Content-Type': 'application/vnd.sas.metadata.instance.query+json',
  'Accept': 'application/vnd.sas.metadata.instance.archive+json',
  'Authorization': 'Bearer ' + access_token
}
response = requests.request("POST", url, headers=headers, data=payload, verify=False)

print(response.text)


# Optional Discovery Agent Requests

## Get Discovery Agent History

This request returns the Agents execution history.

In [ ]:
url = sasserver + "/catalog/bots/" + bot_id + "/history"

payload = {}
headers = {
  'Accept': 'application/vnd.sas.collection+json',
  'Accept-Item': 'application/vnd.sas.metadata.bot.history+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

print(response.text)


## Get Discovery Agent

In [ ]:
url = sasserver + "/catalog/bots/" + bot_id

payload = {}
headers = {
  'Accept': 'application/vnd.sas.metadata.scheduled.bot+json',
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)
print(response)
e_tag = response.headers["ETag"]
print ('ETag='+ e_tag)
response = response.json()
print(response)    


## Update Discovery Agent

This code snippet can be used to update fields or the jobParameters in the Discovery Agent Body.

In [ ]:
url = sasserver + "/catalog/bots/" + bot_id

payload = json.dumps({
  "provider": "TABLE-BOT",
  "name": "Samples Discovery Agent",
  "id": bot_id,
  "description": "Ingest, analyze, and store metadata from Samples caslib",
  "parameters": {
    "datasourceURI": "/dataSources/providers/cas/sources/cas-shared-default~fs~Samples",
    "region": "US NC"
  },
  "jobParameters": {
    "getNLPSemanticID": "0",
    "identifyLanguage": "0",
    "mineForKeywords": "0"
  }
})
headers = {
  'If-Match': e_tag,
  'Accept': 'application/vnd.sas.metadata.bot+json',
  'Content-Type': 'application/json',
  'Authorization': 'Bearer ' + access_token 
}

response = requests.request("PUT", url, headers=headers, data=payload, verify=False)

print(response.text)


## Delete Discovery Agent

In [ ]:
url = sasserver + "/catalog/bots/" + bot_id
payload = {}
headers = {
  'Authorization': 'Bearer ' + access_token
}

response = requests.request("DELETE", url, headers=headers, data=payload, verify=False)
print(response)
